# Initial local machine setup

## Install and import required modules

In [ ]:
!pip3 install wget pycryptodome jumpssh
import os
import sys
from jumpssh import SSHSession # to automate script runs over Jumphost
from shutil import which # to find/verify terraform availability
from Crypto.PublicKey import RSA # to generate key pair

## Setup the environment variables

In [ ]:
# Project variables
username        = ''
project_id      = ''
private_key_name = '' # Don't use file/folder structure - should be in cwd
public_key_name = ''
install_ad_server = False

# Set replacement vars
homepath = '/home/' + username
epicurl = ""
installfile = 'bluedata-epic-entdoc-minimal-release-5.0-3008.bin'
hcpdomain = 'internal.cloudapp.net' # Default domain name used by local Azure DNS

## Azure credentials
subscription_id = ''
client_id       = ''
client_secret   = ''
tenant_id       = ''
region          = ''

%run -i 'my_secrets.py' ### load vars from local file

if not (username, project_id, private_key_name, public_key_name, epicurl, subscription_id, client_id, client_secret, tenant_id, region):
    raise SystemError('You should fill in all the details')
else:
    print('All variables are set')

## Generate OpenSSH keys if needed

In [ ]:
if not os.path.isfile(private_key_name) or not os.path.isfile(public_key_name):
    key = RSA.generate(2048)
    with open(os.open(private_key_name, os.O_CREAT | os.O_WRONLY, 0o600), 'wb') as f:
        f.write(key.exportKey('PEM'))

    pubkey = key.publickey()
    with open(public_key_name, 'wb') as f:
        f.write(pubkey.exportKey('OpenSSH'))
else:
    print('Using existing key files')

## Get terraform

In [ ]:
tf = which('terraform')
if not tf:
    print('Need to download terraform!')
    if sys.platform.startswith('darwin'):
        tf = 'https://releases.hashicorp.com/terraform/0.12.24/terraform_0.12.24_darwin_amd64.zip'
    elif sys.platform.startswith('linux'):
        tf = 'https://releases.hashicorp.com/terraform/0.12.24/terraform_0.12.24_linux_amd64.zip'
    elif sys.platform.startswith('win32'):
        tf = 'https://releases.hashicorp.com/terraform/0.12.24/terraform_0.12.24_windows_amd64.zip'
    tempfile = 'temp.zip'
    import wget
    tfzip = wget.download(tf)
    from zipfile import ZipFile
    with ZipFile(tfzip, 'r') as zip_ref:
        if sys.platform.startswith('win32'):
            zip_ref.extract('terraform.exe')
        else:
            zip_ref.extract('terraform')
    os.remove(tfzip)
    tf = os.path.join('.', 'terraform')

if tf:
    print('Ready to proceed with {}'.format(tf)) 
else:
    raise SystemExit('Please download terraform: https://learn.hashicorp.com/terraform/getting-started/install.html')

## Initialize terraform

In [ ]:
!terraform init

# Pre-flight


## Update scripts

### Import keys

In [ ]:
user_public_key = open(public_key_name, 'r').read().strip()
user_private_key = open(private_key_name, 'r').read().strip()

if not (user_public_key or user_private_key):
    raise SystemExit('You need ssh key pair to proceed')
else:
    print(user_public_key)
    print(user_private_key)


### Update cloud-init

In [ ]:
indent = '  ' # double space for indentation in yaml file
cloudinit = open('cloud-init.yaml-template').read().replace(
    'ssh-authorized-keys:', 'ssh-authorized-keys:\n' + (2 * indent) + '- ' + user_public_key
).replace(
    'name:', 'name: ' + username
)

with open('cloud-init.yaml', 'w') as f:
    f.write(cloudinit)

print (cloudinit)

### Update terraform.tfvars

In [ ]:
tfvars = open('terraform.tfvars-template', 'r').read().replace(
    'subscription_id = ""', 'subscription_id = "' + subscription_id + '"'
).replace(
    'client_id       = ""', 'client_id       = "' + client_id + '"'
).replace(
    'client_secret   = ""', 'client_secret   = "' + client_secret + '"'
).replace(
    'tenant_id       = ""', 'tenant_id       = "' + tenant_id + '"'
).replace(
    'region = ""', 'region = "' + region + '"'
).replace(
    'project_id         = ""', 'project_id         = "' + project_id + '"'
).replace(
    'user               = ""', 'user               = "' + username + '"'
).replace(
    'ssh_pub_key_path   = ""', 'ssh_pub_key_path   = '+ repr(public_key_name).replace('\'', '"')
)

with open('terraform.tfvars', 'w') as f:
    f.write(tfvars)

print(tfvars)

# Ready to Deploy

## Deployment Plan

In [ ]:
!terraform plan -out plan.out

## Run the Deployment

In [ ]:
!terraform apply -auto-approve "plan.out"

# Post Deployment

At this stage, we should have all resources available to start configuring and installing HPE Container Platform

## Get outputs from Terraform

In [ ]:
def get_tfoutput(param):
    tfo = !terraform output $param
    return tfo.s

jumphost_ssh_command = get_tfoutput('jumphost_ssh_command').replace(
    'ssh', 'ssh -i ' + private_key_name
)
connection_info = jumphost_ssh_command # temprorarily store until xrdp is ready

user_password = get_tfoutput('jumphost_password')
jumphost_public_ip = get_tfoutput('jumphost_public_ip')
jumphost_public_dns_name = get_tfoutput('jumphost_public_dns_name')
controller_private_ip = get_tfoutput('controller_private_ip')
jumphost_private_ip = get_tfoutput('jumphost_private_ip')
gateway_private_ip = get_tfoutput('gateway_private_ip')
certfile = 'cacert.pem'
certkeyfile = 'cakey.pem'
import re
worker_private_ips = re.sub('\[|\]|\s|"', '', get_tfoutput('worker_private_ips')).replace(',', ' ').strip().split(' ')

## Configure Jumphost for RDP access, local CA and optional AD service

In [ ]:
jumphost_init_file = os.path.join('scripts', 'jumphost-init.sh')
jumphost_init_script = open(os.path.join('scripts', 'jumphost_init.sh-template')).read().strip()

jumphost_init_script = jumphost_init_script.replace('echo pass', 'echo ' + user_password + ' | sudo passwd --stdin ' + username)

# Install minica
jumphost_init_script = jumphost_init_script.replace(
    'hcp_domains', ','.join([hcpdomain, jumphost_public_dns_name, 'localhost'])
).replace(
    'hcp_ips', ','.join([jumphost_public_ip, jumphost_private_ip, controller_private_ip, gateway_private_ip, *(worker_private_ips), '127.0.0.1'])
).replace(
    'certfile', certfile
).replace(
    'keyfile', certkeyfile
)

# Install AD Server
if install_ad_server:
    jumphost_init_script += '\necho AD server installation not implemented!'

with open(jumphost_init_file, 'w') as f:
    f.write(jumphost_init_script)

print(jumphost_init_script)

## Prepare bluedata_install.sh script

In [ ]:
bluedata_install_script = open(os.path.join('scripts', 'bluedata_install.sh-template')).read().strip()
bluedata_install_script_file = os.path.join('scripts', 'bluedata_install.sh')

# Replace vars
bluedata_install_script = bluedata_install_script.replace(
    '/homepath', homepath
).replace(
    'epicurl', epicurl
).replace(
    'installfile', installfile
).replace(
    'certfile', certfile
).replace(
    'certkeyfile', certkeyfile
).replace(
    'controller_ip', controller_private_ip
).replace(
    'worker_ips', ','.join([*(worker_private_ips)])
).replace(
    'hcpdomain', project_id + '.local'
)

with open(bluedata_install_script_file, 'w') as f:
    f.write(bluedata_install_script)

print(bluedata_install_script)

## Pre-flight for script runs

Allow 2-3 minutes for cloud-init to finish updating packages and rebooting.

In [ ]:
## Check output
def printExit(exit_code, output):
    if exit_code != 0:
        raise SystemExit(output)
    else:
        print(output)

## connect to jumphost
jumphost_ssh_session = SSHSession(jumphost_public_ip, username=username, private_key_file=private_key_name).open()

## Upload private key if not there
if not jumphost_ssh_session.exists(private_key_name):
    jumphost_ssh_session.put(private_key_name, private_key_name, permissions='600')
    print('Private key uploaded to jumphost...')
print('Jumphost ready to connect to other nodes')

## Init jumphost

In [ ]:
(exit_code, jumphost_install_log) = jumphost_ssh_session.run_cmd(jumphost_init_script, continuous_output=True)
if exit_code == 0:
    jumphost_ssh_session.file('jumphost_install.log', jumphost_install_log, permissions='644')
    connection_info = '\nConnect using SSH\n{}\n\nConnect using RDP\nIP: {}\nUsername: {}\nPassword: {}\n'.format(jumphost_ssh_command, jumphost_public_ip, username, user_password)
else:
    raise SystemExit(jumphost_install_log)

## Init all hosts (controller, gateway and workers)

In [ ]:
## worker connections to jumphost
for host in [controller_private_ip, gateway_private_ip, *worker_private_ips]:
    print(host + ' initialization starting')
    host_session = jumphost_ssh_session.get_remote_session(host, username=username, private_key_file=private_key_name)
    host_session.put(private_key_name, private_key_name, permissions='600')
    print('Private key uploaded to host ' + host)
    host_init_file = open(os.path.join('scripts', 'host-init.sh')).read().strip()
    host_session.file('host-init.sh', host_init_file, permissions='755')
    (exit_code, host_init_log) = host_session.run_cmd('./host-init.sh', continuous_output=True)
    jumphost_ssh_session.file('host-' + host + '_init.log', host_init_log, permissions='644')


## Init controller

In [ ]:
## controller connection from jumphost
controller_ssh_session = jumphost_ssh_session.get_remote_session(controller_private_ip, username=username, private_key_file=private_key_name)

## Upload install script
controller_ssh_session.file('bluedata_install.sh', bluedata_install_script, permissions='755')
print('Installation script uploaded to controller...')

## Transfer certificate files to the controller
jumphost_ssh_session.get(hcpdomain + '/cert.pem', certfile)
jumphost_ssh_session.get(hcpdomain + '/key.pem', certkeyfile)
controller_ssh_session.put(certfile, certfile, permissions='600')
controller_ssh_session.put(certkeyfile, certkeyfile, permissions='600')


## Start BlueData installation

In [ ]:
## Start installation
(exit_code, bluedata_install_log) = controller_ssh_session.run_cmd('sudo ./bluedata_install.sh', continuous_output=True)
if exit_code == 0:
    jumphost_ssh_session.file('bluedata_install.log', bluedata_install_log, permissions='644')
else:
    raise SystemExit(bluedata_install_log)

## Clean up

In [ ]:
controller_ssh_session.close()
jumphost_ssh_session.close()

## Ready to serve!

In [ ]:
print(connection_info)

# Post Installation

Follow the steps to complete installation

In [ ]:
gateway_public_dns_name = get_tfoutput('gateway_public_dns_name')
print(open(os.path.join('scripts', 'install_complete-template.txt')).read().strip().replace(
    'private_key_name', private_key_name
).replace(
    'controller_private_ip', controller_private_ip
).replace(
    'worker_private_ips', ','.join(worker_private_ips)
).replace(
    'gateway_private_ip', gateway_private_ip
).replace(
    'gateway_dns_name', gateway_public_dns_name
))

### TODO: Adding gateway

In [ ]:
add_gateway_script = ''


### TODO: Adding worker nodes

In [ ]:
add_workers_script = ''

# Destroy Deployment

Run command `terraform destroy --auto-approve` to automatically destroy the whole deployment.

# TODO

- Add AzureAD and NFS server options
- Disable firewall ports except for gateway (https) and controller (ssh)
- Enable adding gateway & worker nodes